In [5]:
using Base.Test
using BenchmarkTools

# Copying

In [14]:
struct MyStruct
    X::Array
end
A = rand(4,4)
x = MyStruct(A)
@show x.X === A  # "===" tests for memory equality (they point to the same array)
B = x.X
@show A === B;

x.X === A = true
A === B = true


Basically, whenever you use "=" it will point to the same place in memory, even for elements of structs.

Now, how do we "copy" data from one array to another without changing the pointer?

In [31]:
C = rand(4,4) # data we want to copy
B = deepcopy(C)
@show B === A
# Clearly this doesn't work
# let's reassign B to A
B = A
@show A === B
@show B === x.X

# Correct way:
B .= C
@show B === A
@show B === x.X;

# Another way
copy!(B,C)
@show B === A
@show B === x.X

B === A = false
A === B = true
B === x.X = true
B === A = true
B === x.X = true
B === A = true
B === x.X = true


true

Note: This does not work on integers (since dot indexing doesn't make sense in that case)

# Concatenation
Avoid concatenation, especially in time-critical code. It is better to allocate memory and fill it in than to concatenate arrays. 
```
# Example (Pendulum Dynamics): This is slow
return [x[2]; (u - m*g*lc*sin(x[1]) - b*x[2])];

# This is about 3x faster
xdot = zeros(x)
xdot[1] = x[2]
xdot[2] = u[1] - m*g*lc*sin(x[1]) - b*x[2]
return xdot
```
Note that `zeros(x)` is used to create an array of generic type, which is useful for dynamics functions which need to work with `ForwardDiff.Dual` types

# Nested Functions
Basically, nested functions don't lead to any overhead. Take for instance this function `wrapper` that returns another function that is dependent on it's input. Once compiled (the first time that's relative slow), the operation is very fast. Interestingly, we can pull out another function from `wrapper` that behaves differently but still operates under the same compilation of the first (notice no time decrease for the first time `f4` is called.

In [15]:
function wrapper(x)
    function inner(u)
        x-u
    end
end
f = wrapper(2)
@time f(3)
@time f(3)
f4 = wrapper(4)
@time f4(3)

  0.002902 seconds (175 allocations: 10.981 KiB)
  0.000002 seconds (4 allocations: 160 bytes)
  0.000002 seconds (4 allocations: 160 bytes)


1

Now for the clincher, we can define a function that does not have any dependence on the outside function and it has exactly the same performance! Even better, it's performance is identical to the simple operation we're trying to perform. In other words, nested function result in minimal to no overhead!

In [14]:
function wrapper2()
    function inner(u)
        2-u
    end
end
f = wrapper2()
f(3)
@time f(3)
@time 2-3

  0.000002 seconds (4 allocations: 160 bytes)
  0.000002 seconds (4 allocations: 160 bytes)


-1

# Smart Initialization
Defaults for subsequent arguments can depend on previous ones

In [17]:
function myfun(A,b=zeros(size(A,1));C=zeros(A))
    println(C)
    return b
end
A = rand(4,3)
myfun(A)

[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

If we ever want to use default values of another function, we can pass the keyword arguments of one function directly into another.

In [22]:
function inner(a, b; name="noname",age=Inf)
    println("My name is $name")
    println("I'm $age years old")
end
function top(a; kwargs...)
    inner(a,2; kwargs...)
end
top(1,name="brian",age=110)
println()
top(1,name="brian")

My name is brian
I'm 110 years old

My name is brian
I'm Inf years old


Note that if we pass in an argument to the outer function that is not one of the keyword arguments of the inner function we will get an error

In [23]:
top(1,name="brian",ssn=123)

LoadError: [91mMethodError: no method matching inner(::Int64, ::Int64; name="brian", ssn=123)[0m
Closest candidates are:
  inner(::Any, ::Any; name, age) at In[22]:2[91m got unsupported keyword argument "ssn"[39m[39m

# Pass in Dictionary of kwargs
Make sure to use a semi-colon

In [35]:
kwargs = Dict(:name=>"brian",:age=>14)
kwargs[:age] = 23
top(1; kwargs...)

My name is brian
I'm 23 years old


In [40]:
d = Dict{Symbol,Any}()
top(1; d...)

My name is noname
I'm Inf years old


# Push Performance
It's faster to allocate an array and fill it than to append. However, if you allocate too much it's better to use push (this is pretty obvious). Also make sure to specify the type of the vector that you're pushing too.

In [9]:
function pushfill()
    a = Vector{Float64}()
    for i = 1:100
        push!(a,i+0.1)
    end
end
@btime pushfill()

function allocatefill()
    a = ones(1000)
    for i = 1:100
        a[i] = i+0.1
    end
end
@btime allocatefill()


  1.236 μs (107 allocations: 3.83 KiB)
  869.267 ns (1 allocation: 7.94 KiB)


# Multiple Outputs
If you have more than one output and only request one, it will always be of type `Tuple`.
If you request less than the total number of outputs (but greater than 1) the remaining outputs will be discarded.

In [21]:
twoout() = 1,2
a = twoout()
@show typeof(a)

threeout() = 1,2,3
b,c = threeout()  # The `3` value was never stored (all three are actually stored temporarily in `ans`)
@show typeof(c)
d = threeout()
@show typeof(d)

typeof(a) = Tuple{Int64,Int64}
typeof(c) = Int64
typeof(d) = Tuple{Int64,Int64,Int64}


Tuple{Int64,Int64,Int64}